# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Embedded-in-Jupyter-notebook" data-toc-modified-id="Embedded-in-Jupyter-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Embedded in Jupyter notebook</a></div><div class="lev1 toc-item"><a href="#Using-GTK" data-toc-modified-id="Using-GTK-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using GTK</a></div>

# Embedded in Jupyter notebook

In [ ]:
%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt

import time
import threading

import ipywidgets as ipw
import numpy as np
import pandas as pd


fig, axis = plt.subplots()

axis._get_lines()
stop_event = threading.Event()
np.random.seed(0)
data = []


def _draw():
    while True:
        if data:
            pd.Series(np.concatenate(data)).plot(ax=axis)
            fig.canvas.show()
        data.append(np.random.rand(10))
        if stop_event.wait(.5):
            break
    stop_event.clear()
    start.disabled = False
    stop.disabled = True
        
    
def _start(*args):
    start.disabled = True
    stop.disabled = False
    thread = threading.Thread(target=_draw)
    thread.daemon = True
    thread.start()

start = ipw.Button(description='Start')
start.on_click(_start)
stop = ipw.Button(description='Stop')
stop.on_click(lambda *args: stop_event.set())
clear = ipw.Button(description='Clear')
def _clear(*args):
    axis.cla()
    for i in xrange(len(data)):
        data.pop()
clear.on_click(_clear)

widget = ipw.HBox([start, stop, clear])
widget

------------------------------------------------------------------------

# Using GTK

In [ ]:
# See [here][1] for info on handling mouse events in the
# GTK matplotlib interactive widget.
#
# See [here][2] for general usage info for the GTK matplotlib
# interactive widget.
#
# **Note**: Use right-click and drag while in "pan" mode to zoom.
#
# [1]: https://matplotlib.org/users/event_handling.html
# [2]: https://matplotlib.org/users/navigation_toolbar.html
import gobject
import gtk

from matplotlib.figure import Figure
from matplotlib.backends.backend_gtkagg import FigureCanvasGTKAgg as FigureCanvas
from matplotlib.backends.backend_gtkagg import NavigationToolbar2GTKAgg as NavigationToolbar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import threading
import datetime as dt
from pygtkhelpers.delegates import SlaveView
from serial_device.or_event import OrEvent

import matplotlib as mpl
import matplotlib.ticker
import si_prefix as si

A_formatter = mpl.ticker.FuncFormatter(lambda x, *args: '%sA' %
                                       si.si_format(x, 2))
s_formatter = mpl.ticker.FuncFormatter(lambda x, *args: '%ss' %
                                       si.si_format(x))


class StreamingPlot(SlaveView):
    def __init__(self, data=None):
        if data is not None:
            self.data = data
        else:
            self.data = []
        self.data_ready = threading.Event()
        self.stop_event = threading.Event()
        self.line = None
        super(StreamingPlot, self).__init__()
        
    def create_ui(self):
        def _destroy(*args):
            self.stop_event.set()
            
        self.widget.connect('destroy', _destroy)

        vbox = gtk.VBox()
        self.widget.add(vbox)

        self.fig, self.axis = plt.subplots()

        canvas = FigureCanvas(self.fig)  # a gtk.DrawingArea
        vbox.pack_start(canvas)
        toolbar = NavigationToolbar(canvas, win)
        vbox.pack_start(toolbar, False, False)
        self.stop_event.clear()
        np.random.seed(0)
        self.axis.xaxis.set_major_formatter(s_formatter)
        self.axis.yaxis.set_major_formatter(A_formatter)

        self.start_button = gtk.Button('Start')
        self.start_button.connect("clicked", lambda *args: self.start())

        self.stop_button = gtk.Button('Pause')
        self.stop_button.connect("clicked", lambda *args: self.stop_event.set())
        self.stop_button.props.sensitive = False

        self.clear_button = gtk.Button('Reset')
        def _clear(*args):
            self.line = None
            for line_i in self.axis.get_lines():
                line_i.remove()
            for i in xrange(len(self.data)):
                self.data.pop()
            def _reset_ui(*args):
                self.data_ready.clear()
                self.start_button.set_label('Start')
                self.clear_button.props.sensitive = False
                self.fig.canvas.draw()
            gobject.idle_add(_reset_ui)
        self.clear_button.connect("clicked", _clear)
        self.clear_button.props.sensitive = False

        button_box = gtk.HBox()
        for widget_i in (self.start_button, self.stop_button,
                         self.clear_button):
            button_box.pack_start(widget_i, False, False)
        vbox.pack_start(button_box, False, False)

        self.axis.set_ylabel('Current')
        self.axis.set_xlabel('Time')

    def start(self):
        def _draw():
            self.stop_event.clear()
            wait_event = OrEvent(self.stop_event, self.data_ready)
            while True:
                wait_event.wait()
                if self.data_ready.is_set():
                    self.data_ready.clear()
                    plot_data = pd.concat(self.data)
                    absolute_times = plot_data.index.to_series()
                    relative_times = ((absolute_times - absolute_times.iloc[0])
                                      .dt.total_seconds())
                    plot_data.index = relative_times
                    if self.line is None:
                        self.line = self.axis.plot(plot_data.index.values,
                                                   plot_data.values)[0]
                    else:
                        self.line.set_data(plot_data.index.values,
                                           plot_data.values)

                    def _draw_i(axis, plot_data):
                        axis.relim()
                        axis.set_xlim(right=plot_data.index[-1])
                        axis.set_ylim(auto=True)
                        axis.autoscale_view(True, True, True)
                        axis.get_figure().canvas.draw()
                    gobject.idle_add(_draw_i, self.axis, plot_data)
                if self.stop_event.is_set():
                    break

            def _button_states():
                self.start_button.set_label('Continue')
                self.start_button.props.sensitive = True
                self.clear_button.props.sensitive = True
                self.stop_button.props.sensitive = False
            gobject.idle_add(_button_states)

        def _button_states():
            self.start_button.props.sensitive = False
            self.clear_button.props.sensitive = False
            self.stop_button.props.sensitive = True
        gobject.idle_add(_button_states)

        # Start thread to wait for data and plot it when available.  Also
        # listen for `self.stop_event` and stop thread when it is set.
        thread = threading.Thread(target=_draw)
        thread.daemon = True
        thread.start()
        
        delta_t = dt.timedelta(seconds=.1)
        samples_per_plot = 5
        
        def _generate_data():
            while True:
                time_0 = dt.datetime.now()
                values_i = np.random.rand(samples_per_plot)
                absolute_times_i = pd.Series([time_0 + i * delta_t
                                              for i in xrange(len(values_i))])
                data_i = pd.Series(values_i, index=absolute_times_i)
                self.data.append(data_i)
                self.data_ready.set()
                if self.stop_event.wait(samples_per_plot *
                                        delta_t.total_seconds()):
                    break
        # Start thread to generate data and set `self.data_ready` event
        # whenever new data is available.
        # Also Listen for `self.stop_event` and stop thread when it is set.
        data_thread = threading.Thread(target=_generate_data)
        data_thread.daemon = True
        data_thread.start()
        
    
    def on_resize(self):
        def _tight_layout(*args):
            try:
                self.fig.tight_layout()
            except ValueError:
                pass
        gobject.idle_add(_tight_layout)
        
        
with mpl.style.context('fivethirtyeight', {'image.cmap': 'gray',
                                           'image.interpolation' : 'none'}):
    win = gtk.Window()
    win.set_default_size(800, 600)
    view = StreamingPlot()
    win.add(view.widget)
    win.connect('check-resize', lambda *args: view.on_resize())
    win.set_position(gtk.WIN_POS_MOUSE)
    win.show_all()
    view.fig.tight_layout()
    win.connect('destroy', gtk.main_quit)

    gtk.gdk.threads_init()
    gtk.gdk.threads_enter()
    gtk.main()
    gtk.gdk.threads_leave()

In [ ]:
display(*view.data[-3:])

In [ ]:
from IPython.display import display
import bz2


data = pd.concat(view.data)
data_json = data.to_json()
data_json_bz2 = bz2.compress(data_json)
data_from_json = pd.read_json(bz2.decompress(data_json_bz2), typ='series')
len(data_json), len(data_json_bz2)